In [1]:
%load_ext autoreload
%autoreload 2

# Start

In [75]:
import random


from collections import Counter
from engine.player import Player
from engine.character import Character
from engine.game_manager import GameManager

# Game Manager

In [16]:
characters = []
current_day = 0
is_game_over = False
winner = None
role_pool = ['Wolf']*3 + ['Villager']*3 + ['Seer', 'Witch', 'Hunter', 'Idiot']

In [17]:
for i, role_name in enumerate(role_pool):
    # 这里会实例化你之前定义的 Player 类
    player = Player(player_id=i, role_name=role_name)
    character = Character(role_name, player)
    characters.append(character)

## _initialize_beliefs

In [18]:
players_count = len(role_pool)
all_role_counts = Counter(role_pool)
all_role_counts

Counter({'Wolf': 3,
         'Villager': 3,
         'Seer': 1,
         'Witch': 1,
         'Hunter': 1,
         'Idiot': 1})

In [34]:
observer = characters[4]
ob_player = observer.player
ob_role = observer.role
observer.role_name

'Villager'

In [35]:
for target in characters:
    tar_role_name = target.role_name
    tar_player = target.player
    if ob_player.player_id == tar_player.player_id:
        # 对自己的认知：100% 确定
        ob_player.beliefs[tar_player.player_id] = {role: (1.0 if role == observer.role_name else 0.0) for role in all_role_counts}
        continue

    probs = {role: 0.0 for role in all_role_counts}
    if observer.role_name == 'Wolf':
        if target.role_name == 'Wolf':
            # 狼人认队友：100% 确定是狼
            probs['Wolf'] = 1.0
        else:
            # 狼人看其他人：模糊处理（知道不是狼，但不知道具体是哪个好人）
            good_role_count = players_count-all_role_counts['Wolf']
            for role in all_role_counts:
                if role == 'Wolf':
                    continue
                probs[role] = all_role_counts[role]/good_role_count

    else:
        # 好人看其他人：完全模糊（除了自己，剩下11个人都有可能是任何角色）
        # 假设好人视角下的剩余人数分配
        remaining_players_count = players_count-1
        remaining_role_count = all_role_counts.copy()
        remaining_role_count[observer.role_name] -= 1
        for role in all_role_counts:
            probs[role] = remaining_role_count[role]/remaining_players_count
    ob_player.beliefs[tar_player.player_id] = probs

In [36]:
observer.player.beliefs

{0: {'Wolf': 0.3333333333333333,
  'Villager': 0.2222222222222222,
  'Seer': 0.1111111111111111,
  'Witch': 0.1111111111111111,
  'Hunter': 0.1111111111111111,
  'Idiot': 0.1111111111111111},
 1: {'Wolf': 0.3333333333333333,
  'Villager': 0.2222222222222222,
  'Seer': 0.1111111111111111,
  'Witch': 0.1111111111111111,
  'Hunter': 0.1111111111111111,
  'Idiot': 0.1111111111111111},
 2: {'Wolf': 0.3333333333333333,
  'Villager': 0.2222222222222222,
  'Seer': 0.1111111111111111,
  'Witch': 0.1111111111111111,
  'Hunter': 0.1111111111111111,
  'Idiot': 0.1111111111111111},
 3: {'Wolf': 0.3333333333333333,
  'Villager': 0.2222222222222222,
  'Seer': 0.1111111111111111,
  'Witch': 0.1111111111111111,
  'Hunter': 0.1111111111111111,
  'Idiot': 0.1111111111111111},
 4: {'Wolf': 0.0,
  'Villager': 1.0,
  'Seer': 0.0,
  'Witch': 0.0,
  'Hunter': 0.0,
  'Idiot': 0.0},
 5: {'Wolf': 0.3333333333333333,
  'Villager': 0.2222222222222222,
  'Seer': 0.1111111111111111,
  'Witch': 0.1111111111111111,
  

In [34]:
all_role_names

{'Hunter', 'Idiot', 'Seer', 'Villager', 'Witch', 'Wolf'}

## get_game_context

In [43]:
game = GameManager()
game.setup_game()

In [44]:
game.get_game_context()

{'alive_players': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 'public_claims': {0: None,
  1: None,
  2: None,
  3: None,
  4: None,
  5: None,
  6: None,
  7: None,
  8: None,
  9: None,
  10: None,
  11: None},
 'sheriff_id': None,
 'current_day': 0}

## night_phase-wolf

In [76]:
context = game.get_game_context()

In [77]:
target_ids = []
wolves = [c for c in game.characters if c.player.is_alive and c.role_name == "Wolf"]
character_obj = wolves[0]
player = character_obj.player

In [78]:
alive_ids = context["alive_player_ids"]
my_beliefs = player.beliefs
targets = [pid for pid in alive_ids if pid != player.player_id 
        and my_beliefs.get(pid, {}).get("Wolf", 0) < 1]
targets

[4, 5, 6, 7, 8, 9, 10, 11]

In [79]:
for pid in targets:
    if my_beliefs.get(pid, {}).get("Seer", 0) >= 1.0:
        print(pid)
for pid in targets:
    if my_beliefs.get(pid, {}).get("Witch", 0) >= 1.0:
        print(pid)
for pid in targets:
    if my_beliefs.get(pid, {}).get("Hunter", 0) >= 1.0:
        print(pid)
for pid in targets:
    if my_beliefs.get(pid, {}).get("Idiot", 0) >= 1.0:
        print(pid)
for pid in targets:
    if context['gold_water_claims'].get(pid, {}):
        print(pid)
for pid in targets:
    if context['silver_water_claims'].get(pid, {}):
        print(pid)
print(random.choice(targets) if targets else None)
        

5


## night_phase-seer

In [97]:
game = GameManager()
game.setup_game()


In [98]:
print(f"--- 第 {game.current_day} 晚开始 ---")
# 记录本晚发生的关键事件
context = game.get_game_context()

# 1. 狼人行动
target_ids = []
wolves = [c for c in game.characters if c.player.is_alive and c.role_name == "Wolf"]
for wolf in wolves:
    target_id = wolf.role.handle_night_action(wolf, context)
    target_ids.append(target_id)
wolves_target = max(set(target_ids), key=target_ids.count)
context['wolves_target'] = wolves_target

--- 第 0 晚开始 ---


In [99]:
seer = [c for c in game.characters if c.player.is_alive and c.role_name == "Seer"][0]
player = seer.player
role = seer.role

In [113]:
alive_ids = context["alive_player_ids"]
my_beliefs = player.beliefs
targets = [
    pid for pid in alive_ids if pid != player.player_id 
    and (pid not in role.valid_ids)
]
wolf_prop = {id: player.beliefs[id]['Wolf'] for id in targets}
target = max(wolf_prop, key=wolf_prop.get)
target

0

## night_phase-witch

In [136]:
game = GameManager()
game.setup_game()

In [137]:
print(f"--- 第 {game.current_day} 晚开始 ---")
# 记录本晚发生的关键事件
context = game.get_game_context()

# 1. 狼人行动
target_ids = []
wolves = [c for c in game.characters if c.player.is_alive and c.role_name == "Wolf"]
for wolf in wolves:
    target_id = wolf.role.handle_night_action(wolf, context)
    target_ids.append(target_id)
wolves_target = max(set(target_ids), key=target_ids.count)
context['wolves_target'] = wolves_target

seer = [c for c in game.characters if c.player.is_alive and c.role_name == "Seer"][0]
seer_target = seer.role.handle_night_action(seer, context)
context['seer_target'] = seer_target

--- 第 0 晚开始 ---


In [ ]:
witch = [c for c in game.characters if c.player.is_alive and c.role_name == "Witch"][0]


In [ ]:
player = witch.player
role = witch.role
my_beliefs = player.beliefs

heal_target = None
poison_target = None

In [ ]:
if not role.used_heal:
    my_beliefs[context['wolves_target']]['Wolf']=0
    role.used_heal = True
    heal_target = context['wolves_target']
elif not role.used_poison:
    alive_ids = context["alive_player_ids"]
    targets = [
        pid for pid in alive_ids if pid != player.player_id 
    ]
    wolf_prop = {id: player.beliefs[id]['Wolf'] for id in targets}
    poison_target = max(wolf_prop, key=wolf_prop.get)
    role.used_poison = True
else:
    pass

    

{0: 0.36363636363636365,
 1: 0.36363636363636365,
 2: 0.36363636363636365,
 3: 0.36363636363636365,
 4: 0.36363636363636365,
 5: 0.36363636363636365,
 6: 0.36363636363636365,
 7: 0.36363636363636365,
 9: 0.36363636363636365,
 10: 0.36363636363636365,
 11: 0.36363636363636365}

{'Wolf': 0.36363636363636365,
 'Villager': 0.36363636363636365,
 'Seer': 0.09090909090909091,
 'Witch': 0.0,
 'Hunter': 0.09090909090909091,
 'Idiot': 0.09090909090909091}

## night_phase

In [124]:
game = GameManager()
game.setup_game()
context = game.get_game_context()

In [125]:
target_ids = []
wolves = [c for c in game.characters if c.player.is_alive and c.role_name == "Wolf"]
for wolf in wolves:
    target_id = wolf.role.handle_night_action(character_obj=wolf, context=context)
    target_ids.append(target_id)
    

In [126]:
wolves_target = max(set(target_ids), key=target_ids.count)
context['wolves_target'] = wolves_target

In [132]:
seer = [c for c in game.characters if c.player.is_alive and c.role_name == "Seer"][0]
seer_target = seer.role.handle_night_action(seer, context)
context['seer_target'] = seer_target
seer_target_role = game.character_map[seer_target].role_name

In [ ]:
seer.player.beliefs[seer_target] = {k: (1.0 if k == seer_target_role else 0.0) for k in seer.player.beliefs[seer_target]}

{'Wolf': 0.36363636363636365,
 'Villager': 0.36363636363636365,
 'Seer': 0.0,
 'Witch': 0.09090909090909091,
 'Hunter': 0.09090909090909091,
 'Idiot': 0.09090909090909091}

{'Wolf': 1.0,
 'Villager': 0.0,
 'Seer': 0.0,
 'Witch': 0.0,
 'Hunter': 0.0,
 'Idiot': 0.0}